In [7]:
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [9]:
driver = webdriver.Chrome()

### 综合搜索 cpr 女性

In [10]:
driver.get('https://m.weibo.cn/search?containerid=100103type%3D1%26q%3Dcpr%20%E5%A5%B3%E6%80%A7')
time.sleep(3)

In [11]:
# 下拉到底部, 加载所有数据
messages = driver.find_elements(By.XPATH, '//div[@data-v-47035aba=""]')[4:]
prev_len = 0
curr_len = len(messages)

while True:
    if prev_len != curr_len:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        messages = driver.find_elements(By.XPATH, '//div[@data-v-47035aba=""]')[4:]
        prev_len = curr_len
        curr_len = len(messages)
        time.sleep(3)
    else:
        break

In [5]:
df = pd.DataFrame(columns=['id', 'content'])

In [6]:
for message in messages:
    try:
        id = message.find_element(By.XPATH, './/h3[@class="m-text-cut"]').text
    except NoSuchElementException:
        0

    try:
        content = message.find_element(By.XPATH, './/div[@class="weibo-text"]').text
    except NoSuchElementException:
        0

    df.loc[len(df), 'id'] = id
    df.loc[len(df) - 1, 'content'] = content

In [12]:
# 删除df中id重复的行
df.drop_duplicates(subset=['id'], inplace=True)
df.to_csv('weibo.csv', index=False, encoding='utf-8-sig')
df.head(5)

,id,content
0,头条新闻,【#意大利向女性免费提供避孕药#】据界面新闻报道：意媒消息，意大利药品管理局（Aifa）的价...
1,三联生活周刊,【#意大利将向所有女性免费提供口服避孕药#】据界面新闻，据意大利通讯社（AGI）和《信使报》...
2,仲说纷莹,当 CPR 有性别偏见，女性死于心脏骤停 当 CPR 有性别偏见，女性死于心脏骤停
3,圣元爱心园地,【意大利将向所有女性免费提供口服避孕药】据意大利通讯社（AGI）和《信使报》（Il Mess...
4,财富中文网,【#意大利向女性免费提供口服避孕药#】据意媒4月22日消息，意大利药品管理局（Aifa）的价...


In [13]:
driver.close()